## Import Modules

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
from ble_rx_stream import *
import time
import numpy as np
import matplotlib.pyplot as plt

LOG.propagate = False

## Connect to Artemis

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

## Sample Data

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.GET_IMU_TIME, "5000")

In [ ]:
if fut.done():
    s = fut.result()
    stream_data = list(unpack_stream('l6f*', s))
else:
    print('Future not done yet')

In [ ]:
data = np.array(stream_data)
acc_time = data[:, 0]
acc_data = data[:, 1:4]
gyr_data = data[:, 4:7]

In [ ]:
np.mean(np.diff(acc_time))

In [ ]:
_, axs = plt.subplots(2, 1, sharex=True)
axs[0].plot(acc_time, acc_data, label=['x', 'y', 'z'])
axs[0].set_ylabel('acceleration (mg)')
axs[0].legend()
axs[1].plot(acc_time, gyr_data, label=['x', 'y', 'z'])
axs[1].set_ylabel('rotation (deg/s)')
axs[1].legend()
plt.xlabel('time (ms)')
plt.show()

In [ ]:
ble.send_command(CMD.BUF_ALL_TIME, "5000")
time.sleep(5)

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.GET_IMU_BUF, "")

In [ ]:
if fut.done():
    s = fut.result()
    imu_stream_data = list(unpack_stream('l6f*', s))
else:
    print('Future not done yet')

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.GET_TOF_BUF, "1")

In [ ]:
if fut.done():
    s = fut.result()
    tof1_stream_data = list(unpack_stream('lH*', s))
else:
    print('Future not done yet')

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.GET_TOF_BUF, "2")

In [ ]:
if fut.done():
    s = fut.result()
    tof2_stream_data = list(unpack_stream('lH*', s))
else:
    print('Future not done yet')

### Process data

In [ ]:
data = np.array(imu_stream_data)
acc_time = data[:, 0]
acc_data = data[:, 1:4]
gyr_data = data[:, 4:7]
data = np.array(tof1_stream_data)
tof1_time = data[:, 0]
tof1_data = data[:, 1]
data = np.array(tof2_stream_data)
tof2_time = data[:, 0]
tof2_data = data[:, 1]

In [ ]:
_, axs = plt.subplots(3, 1, sharex=True)
axs[0].plot(acc_time, acc_data, label=['x', 'y', 'z'])
axs[0].set_ylabel('acceleration (mg)')
axs[0].legend()
axs[1].plot(acc_time, gyr_data, label=['x', 'y', 'z'])
axs[1].set_ylabel('rotation (deg/s)')
axs[1].legend()
axs[2].plot(tof1_time, tof1_data, label='Sensor 1')
axs[2].plot(tof2_time, tof2_data, label='Sensor 2')
axs[2].set_ylabel('distance (mm)')
axs[2].legend()
plt.xlabel('time (ms)')
plt.show()

## Stunt Recording

### Send command and receive response

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM', True)
ble.send_command(CMD.START_RECORDING_STREAM, "")

In [ ]:
ble.send_command(CMD.STOP_RECORDING_STREAM, "")

In [ ]:
if fut.done():
    s = fut.result()
    stunt_stream_data = list(unpack_stream('l6flHlH*', s))
else:
    print('Future not done yet')

### Process and plot data

In [ ]:
data = np.array(stunt_stream_data)
acc_time = data[:, 0]
acc_data = data[:, 1:4]
gyr_data = data[:, 4:7]
tof1_time = data[:, 7]
tof1_data = data[:, 8]
tof2_time = data[:, 9]
tof2_data = data[:, 10]

In [ ]:
_, axs = plt.subplots(3, 1, sharex=True)
axs[0].plot(acc_time, acc_data, label=['x', 'y', 'z'])
axs[0].set_ylabel('acceleration (mg)')
axs[0].legend()
axs[1].plot(acc_time, gyr_data, label=['x', 'y', 'z'])
axs[1].set_ylabel('rotation (deg/s)')
axs[1].legend()
axs[2].plot(tof1_time, tof1_data, label='Sensor 1')
axs[2].plot(tof2_time, tof2_data, label='Sensor 2')
axs[2].set_ylabel('distance (mm)')
axs[2].legend()
plt.xlabel('time (ms)')
plt.show()

## Disconnect from Artemis

In [ ]:
# Disconnect
ble.disconnect()